In [ ]:
#The code used to scrape new profiles (shown below) looks for profile URLs using Google Search and
#the keywords specified by the user and returns the scraped information from the profiles (name,
#description, location, experience, education and URL, including the data points containing dates and
#locations in the scraping of the experience and education sections).

import csv
import requests
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
from lxml import etree
from parsel import Selector
from flask import request, render_template

def new_profiles():
    experiences = request.form['experiences']
    location = request.form['location']
    studies = request.form['studies']

    # Preparing csv file to store result later
    writer = csv.writer(open('testing.csv', 'w'))
    writer.writerow(['name', 'description', 'location', 'experience', 'education', 'ln_url'])

    driver = webdriver.Chrome('/Users/Gonzalo Rodríguez/chromedriver')
    driver.get('https://www.linkedin.com/login')
    sleep(1)

    # Put your LinkedIn email here
    username_input = driver.find_element("name", 'session_key')
    username_input.send_keys('grodriguez@inveready.com')

    # Put your LinkedIn password here
    password_input = driver.find_element("name", 'session_password')
    password_input.send_keys('password')
    sleep(2)

    # Click on the sign in button
    driver.find_element("xpath", '//button[text()="Iniciar sesión"]').click()
    sleep(2)

    profile_urls = []  # To store the Profile URLs
    text = 'site:linkedin.com/in/ AND "{}" AND "{}" AND "{}"'.format(experiences, location, studies)
    query = 'https://google.com/search?q=' + text
    cookies = {"CONSENT": "YES+shp.gws-20210330-0-RC1.de+FX+412"}
    response = requests.get(query, cookies=cookies)
    soup = BeautifulSoup(response.text, 'html.parser')
    for anchor in soup.find_all('a'):
        url = anchor["href"]
        if 'https://es.linkedin.com/' in url:
            url = url[7:url.find('&')]
            profile_urls.append(url)

    # Visit each profile on LinkedIn and grab details we want to get
    lst = []
    profile_urls1 = ['https://es.linkedin.com/in/julienpalier', 'https://es.linkedin.com/in/eduardgarciaf/en']
    for profile in profile_urls1:
        driver.get(profile)
        sleep(2)
        try:
            soup2 = BeautifulSoup(driver.page_source, 'html.parser')
            dom = etree.HTML(str(soup2))
            sel = Selector(text=driver.page_source)

            # Get name (can get also with xpath)
            try:
                target_name = soup2.find("h1", {"class": "text-heading-xlarge inline t-24 v-align-middle break-words"})
                name = target_name.get_text().strip()
            except:
                name = ''

            # Get Description (can get also with xpath)
            try:
                target_title = soup2.find("div", {"class": "text-body-medium break-words"})
                title = target_title.get_text().strip()
            except:
                title = ''

            # Get location (can get also without xpath)
            try:
                tree = etree.ElementTree(dom)
                find_text = etree.XPath("//div[@class='application-outlet']")
                for target in find_text(dom):
                    loc = tree.getpath(target)
                    location_xpath = loc + '/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[2]/span[1]/text()'
                    location = dom.xpath(location_xpath)[0]
            except:
                location = ''
            location = location.strip()
            ln_url = profile

            # Get Experience
            tree = etree.ElementTree(dom)
            find_text = etree.XPath("//div[@id='experience']")
            for target in find_text(dom):
                pos_exp = tree.getpath(target)
                list1 = [i for i in pos_exp]
                list1[-2] = str(int(pos_exp[-2]) + 2)
                if 'navigation-index-see-all-experiences' in str(soup2):
                    list1.append('/div/div/div/a')
                    pos_exp = ''.join(list1)
                    driver.find_element("xpath", pos_exp).click()
                    sleep(2)
                    soup2 = BeautifulSoup(driver.page_source, 'html.parser')
                    dom = etree.HTML(str(soup2))
                    sel = Selector(text=driver.page_source)
                    sel_exp = str(soup2)
                    num_exp = sel_exp.count('<li class="pvs-list__paged-list-item artdeco-list__item \
                                            pvs-list__item--line-separated pvs-list__item--one-column"')
                    i = 0
                    experiences = []
                    while i < num_exp:
                        i += 1
                        tree = etree.ElementTree(dom)
                        find_text = etree.XPath("//div[@class='application-outlet']")
                        for target in find_text(dom):
                            pos_exp = tree.getpath(target)
                            listi1 = pos_exp + '/div[3]/div/div/div[2]/div/div/main/section/div[2]/div/div[1]/ul/li[' + str(i) + ']'
                            try:
                                sel_li = sel.xpath(listi1).extract()[0]
                                if sel_li.count('li class="pvs-list__paged-list-item pvs-list__item--one-column"') > 1:
                                    # (Code for handling positions with promotions)
                                else:
                                    # (Code for handling positions without promotions)
                            except:
                                pass

                        # Go back if you are inside all experiences
                        driver.execute_script("window.history.go(-1)")
                        sleep(2)
                        soup2 = BeautifulSoup(driver.page_source, 'html.parser')
                        dom = etree.HTML(str(soup2))
                        sel = Selector(text=driver.page_source)

            # Get education
            tree = etree.ElementTree(dom)
            find_text = etree.XPath("//div[@id='education']")
            for target in find_text(dom):
                pos_edu = tree.getpath(target)
                list2 = [i for i in pos_edu]
                list2[-2] = str(int(pos_edu[-2]) + 2)
                if 'navigation-index-see-all-education' in str(soup2):
                    list2.append('/div/div/div/a')
                    pos_edu = ''.join(list2)
                    driver.find_element("xpath", pos_edu).click()
                    sleep(2)
                    soup2 = BeautifulSoup(driver.page_source, 'html.parser')
                    dom = etree.HTML(str(soup2))
                    sel = Selector(text=driver.page_source)
                    sel_edu = str(soup2)
                    num_edu = sel_edu.count('<li class="pvs-list__paged-list-item artdeco-list__item \
                                            pvs-list__item--line-separated pvs-list__item--one-column"')
                    i = 0
                    education = []
                    while i < num_edu:
                        i += 1
                        tree = etree.ElementTree(dom)
                        find_text = etree.XPath("//div[@class='application-outlet']")
                        for target in find_text(dom):
                            pos_edu = tree.getpath(target)
                            listi2 = pos_edu + '/div[3]/div/div/div[2]/div/div/main/section/div[2]/div/div[1]/ul/li[' + str(i) + ']/div/div/div[2]/div[1]/a'
                            # (Code for extracting school, degree, and year from education section)
                else:
                    # (Code for handling education extraction when not going to 'see all education')

            writer.writerow([name, title, location, experiences, education, ln_url])
            lst.append([name, title, location, experiences, education, ln_url])

        except Exception as e:
            print("Error occurred:", e)
            continue

    print(lst)
    print(profile_urls)
    driver.quit()
    return render_template('index.html', lst=lst)
